In [1]:
import cv2
import numpy as np
green_cnt, cnt = 0, 0 # 긴 선분의 개수 세는 것
flag_l, flag_r = False, False #좌우 선분에서 

def region_of_interest(img,vertices): 
    # 이 함수의 경우에는 도움 받음 - gpt, 마스크 처리한 이미지 반환하는 함수
    mask = np.zeros_like(img)          
    match_mask_color = 255 
    cv2.fillPoly(mask,vertices,match_mask_color)
    masked_image=cv2.bitwise_and(img,mask) 
    return masked_image
  
def draw_the_lines(img,lines):
  imge=np.copy(img)     
  global green_cnt, flag_l, flag_r,cnt
  green_cnt = 0
  cnt = 0 #green_cnt 초기화
  flag_r, flag_l = False, False # flag들 초기화
  blank_image=np.zeros((imge.shape[0],imge.shape[1],3),dtype=np.uint8)
  if lines is None:
      return imge
  
  #line들에 대해서 길이를 측정하고 그리기, 길이에 따라 유형 구분
  for line in lines:  
    for x1,y1,x2,y2 in line:
        # print(x1,x2,y1,y2)
        color = (255,0,0)
        cnt += 1
        #꺽이는지 여부에 따라서 다른 색을 표현하고자 함
        if (abs(x1-x2) / img.shape[1] < abs(y1-y2) / img.shape[0]):
            color = (0,255,0)
            green_cnt += 1
            if x1 < img.shape[1] // 2 and x2 < img.shape[1] * 2 //3:
                flag_l = True
            elif x1 > img.shape[1] // 2 and x1 > img.shape[1] // 3:
                flag_r = True
        cv2.line(blank_image,(x1,y1),(x2,y2),color,thickness=20)
        imge = cv2.addWeighted(imge,1,blank_image,1,0.0) 
  return imge

def filter_color(image):
    white_threshold = 200
    lower_white = np.array([white_threshold, white_threshold, white_threshold])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(image, lower_white, upper_white)
    white_image = cv2.bitwise_and(image, image, mask=white_mask)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([90,100,100])
    upper_yellow = np.array([110,255,255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_image = cv2.bitwise_and(image, image, mask=yellow_mask)
    image2 = cv2.addWeighted(white_image, 1., yellow_image, 1., 0.)
    return image2
 

def process_image(image):
    global green_cnt,flag_r,flag_l,cnt
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    height,width=image.shape[0],image.shape[1]
    region_of_interest_coor=[(0,min(height//2,400)),(width,min(height//2,400)),(0,height),(width,height)]
    
    # 이미지에 필터 처리하여서 노이즈 제거
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    #blackhat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, k)
    eroded = cv2.erode(image, k)
    gray_image = cv2.cvtColor(eroded,cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray_image,(11,11),0)
    # blurred = cv2.Laplacian(gray_image,-1)
    canny_image = cv2.Canny(blurred,50,200)
    
    #gpt의 도움 받음 - roi 이미지 선택하고 이에서 허프 변환을 통해 선을 탐지
    cropped = region_of_interest(canny_image,np.array([region_of_interest_coor],np.int32))
    lines = cv2.HoughLinesP(cropped,rho=1,theta=np.pi/180,threshold=15,lines=np.array([]),minLineLength=20,maxLineGap=100)
    image_with_lines = draw_the_lines(image,lines)
    #여기까지만 
    
    # 이미지 합치고 green_cnt의 개수에 따라서 위험 정도 판별
    # 긴 경계의 수가 많으면 그만큼 경계로 둘러싸일 확률이 높다 -> 안전할 확률이 높다
    # print(green_cnt, cnt, flag_l, flag_r)
    combined_image = cv2.addWeighted(image, 0.8, image_with_lines, 1, 1)
    if green_cnt / (cnt+1) < 0.2 and (flag_l == False and flag_r == False):
        cv2.putText(combined_image, "Danger!" , (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 5)
    elif green_cnt / (cnt+1) < 0.5 or (flag_l == False or flag_r == False):
        cv2.putText(combined_image, "Maybe Danger!" , (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 5)
    else:
        cv2.putText(combined_image, "No Bangsim!" , (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 5)
    combined_image = cv2.cvtColor(combined_image,cv2.COLOR_RGB2BGR)
    return combined_image

#비디오 받아서 수행 - 웹캠으로 바꿔서 실전 적용
cap = cv2.VideoCapture('./video8.mp4')
if cap.isOpened():
    while True:
        ret, image = cap.read()
        green_cnt,cnt = 0,0
        if not ret:
            break
        mask_image = filter_color(image)
        processed_image = process_image(image)
        cv2.imshow('Lane Detection', processed_image)
        key = cv2.waitKey(1)
        if key == 27:
            break

cap.release()
cv2.destroyAllWindows()